In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("lab5") \
    .getOrCreate()

sc = spark.sparkContext

Spark Session initialized!
Spark Version: 3.5.3


In [3]:
!wget https://raw.githubusercontent.com/kropiak/uwm_analiza_duzych_zbiorow/refs/heads/main/lab_05/pan-tadeusz.txt

--2024-11-25 14:40:28--  https://raw.githubusercontent.com/kropiak/uwm_analiza_duzych_zbiorow/refs/heads/main/lab_05/pan-tadeusz.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 482907 (472K) [text/plain]
Saving to: ‘pan-tadeusz.txt’

pan-tadeusz.txt     100%[===================>] 471.59K  --.-KB/s    in 0.05s   

2024-11-25 14:40:28 (10.0 MB/s) - ‘pan-tadeusz.txt’ saved [482907/482907]



In [4]:
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

lines_with_tadeusz = pan_tadeusz_file.filter(lambda line: 'tadeusz' in line.lower())
tadeusz_count = lines_with_tadeusz.count()

print(f"Liczba linii zawierających słowo 'Tadeusz': {tadeusz_count}")

Liczba linii zawierających słowo 'Tadeusz': 183


In [7]:
top_3_lines = pan_tadeusz_file.top(3, key=lambda line: len(line))

print("3 najdłuższe linie:\n")
for line in top_3_lines:
    print(line)

3 najdłuższe linie:

O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.
Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem — Kłótnia — Hrabia z Gerwazym odbywają rad

In [8]:
import re
import json
import urllib.request

stopwords_url = "https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt"
stopwords_response = urllib.request.urlopen(stopwords_url)
stopwords = set(stopwords_response.read().decode().splitlines())

def clean_line(line):
    words = re.sub(r'[^\w\s]', '', line.lower()).split()
    return [word for word in words if word not in stopwords and len(word) > 1]

clean_words = pan_tadeusz_file.flatMap(clean_line)
word_counts = clean_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

word_count_dict = word_counts.collectAsMap()
with open("pan_tadeusz_bag_of_words.json", "w", encoding="utf-8") as json_file:
    json.dump(word_count_dict, json_file, ensure_ascii=False, indent=4)

most_common_word = word_counts.takeOrdered(1, key=lambda x: -x[1])[0]
print(f"Najczęściej występujące słowo: '{most_common_word[0]}', występuje {most_common_word[1]} razy.")

Najczęściej występujące słowo: 'rzekł', występuje 155 razy.
